## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Feels Like,Min Temp,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Description,Date
0,0,Oranjemund,-28.5500,16.4333,63.59,63.32,63.32,90,52,6.51,NaN,broken clouds,2021-12-05 20:06:09
1,1,Chokurdakh,70.6333,147.9167,-54.87,-42.27,-42.27,100,61,5.32,RU,broken clouds,2021-12-05 20:06:10
2,2,Lamar,33.6668,-95.5836,73.00,69.78,75.36,78,90,18.41,US,overcast clouds,2021-12-05 20:06:10
3,3,Saint-Philippe,-21.3585,55.7679,77.00,75.22,76.59,80,37,10.31,RE,scattered clouds,2021-12-05 20:06:10
4,4,Kodiak,57.7900,-152.4072,21.58,27.91,27.91,93,20,5.75,US,few clouds,2021-12-05 20:06:11


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?  "))
max_temp = float(input("What is the maximum temperature you would like for your trip?  "))

What is the minimum temperature you would like for your trip?  70
What is the maximum temperature you would like for your trip?  90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp)
                                       & (city_data_df['Max Temp'] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Feels Like,Min Temp,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Description,Date
2,2,Lamar,33.6668,-95.5836,73.00,69.78,75.36,78,90,18.41,US,overcast clouds,2021-12-05 20:06:10
3,3,Saint-Philippe,-21.3585,55.7679,77.00,75.22,76.59,80,37,10.31,RE,scattered clouds,2021-12-05 20:06:10
5,5,Tirumullaivasal,11.2333,79.8333,80.15,78.67,78.67,83,25,6.31,IN,scattered clouds,2021-12-05 20:06:11
7,7,Matara,5.9485,80.5353,77.40,75.83,75.83,91,96,3.58,LK,moderate rain,2021-12-05 20:06:12
8,8,Kavieng,-2.5744,150.7967,89.02,82.09,82.09,80,96,7.85,PG,overcast clouds,2021-12-05 20:06:12
10,10,Faanui,-16.4833,-151.7500,84.81,81.01,81.01,71,46,18.99,PF,scattered clouds,2021-12-05 20:06:13
11,11,Saint George,37.1041,-113.5841,56.50,55.99,70.54,29,1,3.44,US,clear sky,2021-12-05 20:03:03
12,12,Bonfim,3.0833,-59.9500,93.52,89.65,89.65,49,100,8.88,BR,overcast clouds,2021-12-05 20:06:14
17,17,Avarua,-21.2078,-159.7750,87.28,80.65,80.65,89,75,6.91,CK,light rain,2021-12-05 20:06:15
22,22,Kapaa,22.0752,-159.3190,75.92,74.48,77.32,69,1,1.99,US,light rain,2021-12-05 20:06:17


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID        0
City           0
Lat            0
Lng            0
Feels Like     0
Min Temp       0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Country        7
Description    0
Date           0
dtype: int64

In [ ]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
pref_cities_dropna_df = preferred_cities_df.dropna()
pref_cities_dropna_df.count()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = pref_cities_dropna_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
2,Lamar,US,75.36,overcast clouds,33.6668,-95.5836,
3,Saint-Philippe,RE,76.59,scattered clouds,-21.3585,55.7679,
5,Tirumullaivasal,IN,78.67,scattered clouds,11.2333,79.8333,
7,Matara,LK,75.83,moderate rain,5.9485,80.5353,
8,Kavieng,PG,82.09,overcast clouds,-2.5744,150.7967,
10,Faanui,PF,81.01,scattered clouds,-16.4833,-151.7500,
11,Saint George,US,70.54,clear sky,37.1041,-113.5841,
12,Bonfim,BR,89.65,overcast clouds,3.0833,-59.9500,
17,Avarua,CK,80.65,light rain,-21.2078,-159.7750,
22,Kapaa,US,77.32,light rain,22.0752,-159.3190,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 40000,
    "type": "lodging",
    "key": g_key}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    params['locations'] = f'{lat},{lng}'
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    #6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print('Hotel not found...skipping.')
        continue

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...sk

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...sk

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...sk

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...sk

In [9]:
requests.get(base_url, params=params).json()

{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}

In [10]:
hotel_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
2,Lamar,US,75.36,overcast clouds,33.6668,-95.5836,
3,Saint-Philippe,RE,76.59,scattered clouds,-21.3585,55.7679,
5,Tirumullaivasal,IN,78.67,scattered clouds,11.2333,79.8333,
7,Matara,LK,75.83,moderate rain,5.9485,80.5353,
8,Kavieng,PG,82.09,overcast clouds,-2.5744,150.7967,
...,...,...,...,...,...,...,...
2573,Dicabisagan,PH,70.63,overcast clouds,17.0818,122.4157,
2575,Samana,DO,84.42,scattered clouds,19.2056,-69.3369,
2577,Acopiara,BR,87.37,broken clouds,-6.0953,-39.4525,
2579,Bom Jesus Da Lapa,BR,76.91,broken clouds,-13.2550,-43.4181,


In [11]:
# 7. Drop the rows where there is no Hotel Name.
#clean_hotel_df = hotel_df.dropna('Hotel Name')

In [12]:
# 8a. Create the output File (CSV)
#vacation_data_df = clean_hotel_df

# 8b. Export the City_Data into a csv
#vacation_data_df.to_csv('WeatherPy_Vacation.csv', index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code,
# the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
#hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
#locations = clean_hotel_df[["Lat", "Lng"]]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[['Lat', 'Lng']]
max_temp = hotel_df['Max Temp']
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations,
                                 weights=max_temp,
                                 dissipating=False,
                                 max_intensity=300,
                                 point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))